## Main Simulation

In [ ]:
import heapq
import random
import scipy.stats as sts
from operator import attrgetter
import numpy as np

In [ ]:
class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.

    Attributes
    ----------
    <include your list and description here>

    '''

    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        return self.timestamp < other.timestamp

    def run(self, schedule):
        try:
            self.function(schedule, *self.args, **self.kwargs)
        except:
            print('Error running event:')
            # BUG BUG BUG 🐞🐞🐞 PATCHED UP WITH A TRY-EXCEPT 🐞🐞🐞
            print(self.function, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.

    The `now` attribute contains the time at which the last event was run.

    Attributes
    ----------
    <include your list and description here>

    '''

    def __init__(self):
        self.now = 0
        self.priority_queue = []

    def add_event_at(self, timestamp, function, *args, **kwargs):
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))

    def add_event_after(self, interval, function, *args, **kwargs):
        self.add_event_at(self.now + interval, function, *args, **kwargs)

    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        
        event = heapq.heappop(self.priority_queue)        
        self.now = event.timestamp
        event.run(self)

    def __repr__(self):
        return (
                f'Schedule() at time {self.now}min ' +
                f'with {len(self.priority_queue)} events in the queue')

    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'  ⏱ {event.timestamp}min: {event.function.__name__}')



In [85]:


class Queue_MG1:
    def __init__(self, service_distribution, queue_number, manager):

        self.service_distribution = service_distribution
        self.people_in_queue = 0
        self.people_being_served = 0

        self.queue_number = queue_number

        # variables to keep track of the time spent in the queue (future metrics)
        self.start_serving = 0
        self.finish_serving = 0
        

    def add_customer(self, schedule, manager):
        # Add the customer to the queue
        self.people_in_queue += 1
        print(
            f'⏱{schedule.now:5.2f}min: Add customer to queue {self.queue_number}.  '
            f' 🧍🏾‍♀️People in the queue: {self.people_in_queue}')

        if self.people_being_served < 1:
            # This customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer, manager)


    def start_serving_customer(self, schedule, manager):
        # Move the customer from the queue to a server
        self.people_in_queue -= 1
        self.people_being_served += 1
        
        self.start_serving = schedule.now
        print(
            f'⏱{schedule.now:5.2f}min: Start serving customer in queue {self.queue_number}. '
            f' 🧍🏾‍♀️People in the queue number {self.queue_number}: {self.people_in_queue}')
        # Schedule when the server will be done with the customer
        schedule.add_event_after(
            self.service_distribution.rvs(),
            self.finish_serving_customer, manager)

    def finish_serving_customer(self, schedule, manager):
        # Remove the customer from the server
        self.people_being_served -= 1

        self.finish_serving = schedule.now 


        print(
            f'⏱{schedule.now:5.2f}min: Stop serving customer in queue number {self.queue_number}.')
        
        if self.people_in_queue > 0:
            # There are more people in the queue so serve the next customer
            schedule.add_event_after(0, self.start_serving_customer, manager)

        # 5% chance that the person who just got served will join the manager queue
        if random.random() < 0.05:
            self.send_customer_to_manager(schedule, manager)
            schedule.add_event_after(0 + 0.01, self.finish_serving_customer)
    
    def send_customer_to_manager(self, schedule, manager):
        # with a 5% chance likelihood, the person who just got served will join the Manager queue
        # if the manager is not busy, the person will be served immediately
        # if the manager is busy, the person will join the queue
        # the manager's service distribution is the same as the queue's
        # there is only one manager.
        manager.add_customer(schedule)



class Manager():
    # replica of the Queue_MG1 class, but with a different name
    # should be refactored, probably through inheritance
    def __init__(self, service_distribution):
        self.service_distribution = service_distribution
        self.people_in_queue = 0
        self.people_being_served = 0
        self.start_serving = 0
        self.finish_serving = 0

    def add_customer(self, schedule):
        self.people_in_queue += 1
        print( f'⏱{schedule.now:5.2f}min: Add customer to manager queue. '
         f'People in manager queue {self.people_in_queue}')
        
        if self.people_being_served < 1:
            # This customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer)
            
    def start_serving_customer(self, schedule):
        self.people_in_queue -= 1
        self.people_being_served += 1
        self.start_serving = schedule.now

        print( f'⏱{schedule.now:5.2f}min: Manager serving person')
        schedule.add_event_after(self.service_distribution.rvs(), self.finish_serving_customer)

            
    def finish_serving_customer(self, schedule):
        # Remove the customer from the Manager
        self.people_being_served -= 1
        self.finish_serving = schedule.now

        print( f'⏱{schedule.now:5.2f}min: Manager done serving person')
        if self.people_in_queue > 0:
            # There are more people in the queue so serve the next customer
            schedule.add_event_after(0, self.start_serving_customer)
    



class GroceryStore_MG1:
    # not MG1 but MGC? # since it's a system of MG1 queues? Or,  c*MG1?
    def __init__(self, arrival_distribution, service_distribution, manager_serv_distribution, num_queues):

        self.arrival_distribution = arrival_distribution

        # making it from an MG1 queue but really, it's a GG1 queue
        self.manager = Manager(manager_serv_distribution)

        self.queues = {}
        for i in range(num_queues):
            self.queues[i] = Queue_MG1(service_distribution, i, self.manager)

    def add_customer(self, schedule):

        # find the queue with the smallest number of people
        self.min_queue = min(self.queues.values(), key=attrgetter("people_in_queue"))

        # Add this customer to that queue
        self.min_queue.add_customer(schedule, self.manager)
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)

    def run(self, schedule):
        # Schedule the first customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(), 
            self.add_customer)


def run_simulation_MG1(arrival_distribution, service_distribution, manager_serv_distribution, num_queues, run_until):
    schedule = Schedule()
    grocery_store = GroceryStore_MG1(arrival_distribution, service_distribution, manager_serv_distribution, num_queues)
    grocery_store.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
    print(f'Finished running simulation for {run_until} minutes for {num_queues} queues')
    return grocery_store



In [86]:
arrival_distribution = sts.expon(scale=1)
service_distribution = sts.norm(loc=3, scale=np.sqrt(1))
manager_serv_distribution = sts.norm(loc=5, scale=np.sqrt(2))
number_queues = 3
duration = 80

grocery_store = run_simulation_MG1(arrival_distribution, service_distribution, manager_serv_distribution, number_queues, duration)

⏱ 1.17min: Add customer to queue 0.   🧍🏾‍♀️People in the queue: 1
⏱ 1.17min: Start serving customer in queue 0.  🧍🏾‍♀️People in the queue number 0: 0
⏱ 2.87min: Add customer to queue 0.   🧍🏾‍♀️People in the queue: 1
⏱ 2.93min: Add customer to queue 1.   🧍🏾‍♀️People in the queue: 1
⏱ 2.93min: Start serving customer in queue 1.  🧍🏾‍♀️People in the queue number 1: 0
⏱ 4.27min: Add customer to queue 1.   🧍🏾‍♀️People in the queue: 1
⏱ 4.70min: Stop serving customer in queue number 0.
⏱ 4.70min: Start serving customer in queue 0.  🧍🏾‍♀️People in the queue number 0: 0
⏱ 6.37min: Stop serving customer in queue number 0.
⏱ 6.60min: Add customer to queue 0.   🧍🏾‍♀️People in the queue: 1
⏱ 6.60min: Start serving customer in queue 0.  🧍🏾‍♀️People in the queue number 0: 0
⏱ 6.77min: Stop serving customer in queue number 1.
⏱ 6.77min: Start serving customer in queue 1.  🧍🏾‍♀️People in the queue number 1: 0
⏱ 6.87min: Add customer to queue 0.   🧍🏾‍♀️People in the queue: 1
⏱ 7.05min: Add customer to q